In [3]:
pip install matplotlib

     |████████████████████████████████| 7.2 MB 2.0 MB/s 
     |████████████████████████████████| 61 kB 426 kB/s 
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
     |████████████████████████████████| 2.8 MB 9.1 MB/s 
You should consider upgrading via the '/Users/snow/.virtualenvs/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install statsmodels

     |████████████████████████████████| 9.5 MB 970 kB/s 
     |████████████████████████████████| 231 kB 14.1 MB/s 
     |████████████████████████████████| 30.8 MB 9.6 MB/s 
You should consider upgrading via the '/Users/snow/.virtualenvs/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install tqdm

     |████████████████████████████████| 75 kB 2.5 MB/s 
You should consider upgrading via the '/Users/snow/.virtualenvs/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
#In[]
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd
import statsmodels.api as sm
from tqdm import tqdm
from multiprocessing.dummy import Pool
pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)

In [13]:
#In[]
crsp_df = pd.read_csv("crsp.zip", compression='zip',header=0,
             parse_dates = ['date','DCLRDT','RCRDDT'],
             dtype={"PRC": np.float64,
                  "VOL": np.float64,
                  "SHROUT": np.float64,
                  "DIVAMT": np.float64,
                  "SPREAD": np.float64,
                  "SHRCD":str,
                  "DISTCD":str,
                  "COMNAM": str,
                  "TICKER": str,
                   'CUSIP': str})
#Data Preprocessing
crsp_df = crsp_df[(crsp_df.SHRCD.isin(('10','11')))]
crsp_df = crsp_df[(crsp_df.date >= '1927-01-01')]
crsp_df = crsp_df[(crsp_df.date <= '2011-12-31')]
cusips_DISTCD_12 = crsp_df[crsp_df.DISTCD.apply(lambda x: x[:2]=='12' if isinstance(x, str) else False)].CUSIP.unique()
crsp_df = crsp_df[crsp_df.CUSIP.isin(cusips_DISTCD_12)]

crsp_df.PRC = crsp_df.PRC.abs()


crsp_df['year'] = crsp_df['date'].dt.year
crsp_df['month'] = crsp_df['date'].dt.month
crsp_df.sort_values(by=['CUSIP','date'], ascending=True, inplace=True)
crsp_df['PRC_t-1'] = crsp_df.groupby('CUSIP')['PRC'].shift(1)
crsp_df.head()

,PERMNO,date,SHRCD,TICKER,COMNAM,CUSIP,DCLRDT,RCRDDT,DISTCD,DIVAMT,PRC,VOL,RET,SHROUT,SPREAD,RETX,year,month,PRC_t-1
1359100,25881,1970-11-30,10,AE,A & E PLASTIK PAK INC,00003210,NaT,NaT,NaN,NaN,9.000,318.0,C,2655.0,NaN,C,1970,11,NaN
1359101,25881,1970-12-31,10,AE,A & E PLASTIK PAK INC,00003210,NaT,NaT,NaN,NaN,10.000,646.0,0.111111,2655.0,NaN,0.111111,1970,12,9.000
1359102,25881,1971-01-29,10,AE,A & E PLASTIK PAK INC,00003210,NaT,NaT,NaN,NaN,9.000,636.0,-0.100000,2655.0,NaN,-0.100000,1971,1,10.000
1359103,25881,1971-02-26,10,AE,A & E PLASTIK PAK INC,00003210,NaT,NaT,NaN,NaN,11.125,2155.0,0.236111,2655.0,NaN,0.236111,1971,2,9.000
1359104,25881,1971-03-31,10,AE,A & E PLASTIK PAK INC,00003210,NaT,NaT,NaN,NaN,9.750,2062.0,-0.123596,2655.0,NaN,-0.123596,1971,3,11.125


In [14]:
#In[]
crsp_div_df = crsp_df[['date','CUSIP','DCLRDT','RCRDDT','DISTCD','DIVAMT','PRC','VOL','RET','SHROUT','SPREAD','year','month']].copy()
crsp_div_df.sort_values(by=['CUSIP','date'], ascending=True, inplace=True)
crsp_div_df = crsp_div_df.groupby(by=['CUSIP','date']).agg({
     'DCLRDT': 'last',
     'RCRDDT': 'last',
     'DISTCD': 'last',
     'DIVAMT': 'sum',
     'RET':'max',
     'VOL': 'last',
     'PRC': 'last',
     'SPREAD': 'last',
     'SHROUT': 'last'})
crsp_div_df['MCAP'] = crsp_div_df['PRC'] * crsp_div_df['SHROUT']
crsp_div_df

DCLRDT RCRDDT DISTCD  DIVAMT        RET        VOL  \
CUSIP    date                                                            
00003210 1970-11-30    NaT    NaT   None     0.0          C      318.0   
         1970-12-31    NaT    NaT   None     0.0   0.111111      646.0   
         1971-01-29    NaT    NaT   None     0.0  -0.100000      636.0   
         1971-02-26    NaT    NaT   None     0.0   0.236111     2155.0   
         1971-03-31    NaT    NaT   None     0.0  -0.123596     2062.0   
...                    ...    ...    ...     ...        ...        ...   
N5946510 2011-08-31    NaT    NaT   None     0.0  -0.089113  2419635.0   
         2011-09-30    NaT    NaT   None     0.0  -0.181205  1672860.0   
         2011-10-31    NaT    NaT   None     0.0   0.151854  1538552.0   
         2011-11-30    NaT    NaT   None     0.0  -0.002044  1220142.0   
         2011-12-30    NaT    NaT   None     0.0   0.098822  1185432.0   

                        PRC  SPREAD    SHROUT         MCAP  
CUSIP    date                                               
00003210 1970-11-30   9.000     NaN    2655.0    23895.000  
         1970-12-31  10.000     NaN    2655.0    26550.000  
         1971-01-29   9.000     NaN    2655.0    23895.000  
         1971-02-26  11.125     NaN    2655.0    29536.875  
         1971-03-31   9.750     NaN    2655.0    25886.250  
...                     ...     ...       ...          ...  
N5946510 2011-08-31  20.750     NaN  426263.0  8844957.250  
         2011-09-30  16.990     NaN  426485.0  7245980.150  
         2011-10-31  19.570     NaN  426512.0  8346839.840  
         2011-11-30  19.530     NaN  426512.0  8329779.360  
         2011-12-30  21.460     NaN  426678.0  9156509.880  

[2097733 rows x 10 columns]

In [31]:
#In[]
crsp_div_df.sort_values(by=['CUSIP','date'], ascending=True, inplace=True)
crsp_div_df['freq'] = None
crsp_div_df['freq'] = crsp_div_df[crsp_div_df.RCRDDT.isna()==False]['DISTCD'].apply(lambda x: x[2:3] if x is not None else None)
crsp_div_df['freq'] = crsp_div_df.groupby(by=['CUSIP']).fillna(method='ffill', limit=11)['freq']


In [46]:
crsp_div_df['div_yr']=crsp_div_df[crsp_div_df.freq=='6'].apply(lambda x:True )
crsp_div_df['div_semi_annual']=crsp_div_df[crsp_div_df.freq=='4'].apply(lambda x:True )
crsp_div_df['div_quarterly']=crsp_div_df[crsp_div_df.freq=='3'].apply(lambda x:True )
crsp_div_df['div_unknown']=crsp_div_df[crsp_div_df.freq=='0'].apply(lambda x:True )
crsp_div_df['div_monthly']=crsp_div_df[crsp_div_df.freq=='2'].apply(lambda x:True )
crsp_temp=crsp_div_df


In [48]:
crsp_temp['lag_3']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(3)
crsp_temp['lag_6']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(6)
crsp_temp['lag_9']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(9)
crsp_temp['lag_12']=crsp_temp.groupby('CUSIP')['div_unknown'].shift(12)
crsp_temp['lag_quarterly']=crsp_temp.groupby('CUSIP')['div_quarterly'].shift(3)
crsp_temp['lag_semi_annual']=crsp_temp.groupby('CUSIP')['div_semi_annual'].shift(6)
crsp_temp['lag_div_yr']=crsp_temp.groupby('CUSIP')['div_yr'].shift(12)
crsp_temp['lag_1']=crsp_temp.groupby('CUSIP')['div_monthly'].shift(1)
crsp_temp['lag_4']=crsp_temp.groupby('CUSIP')['div_monthly'].shift(4)
crsp_temp['lag_7']=crsp_temp.groupby('CUSIP')['div_monthly'].shift(7)
crsp_temp['lag_10']=crsp_temp.groupby('CUSIP')['div_monthly'].shift(10)


crsp_temp

DCLRDT RCRDDT DISTCD  DIVAMT        RET        VOL  \
CUSIP    date                                                            
00003210 1970-11-30    NaT    NaT   None     0.0          C      318.0   
         1970-12-31    NaT    NaT   None     0.0   0.111111      646.0   
         1971-01-29    NaT    NaT   None     0.0  -0.100000      636.0   
         1971-02-26    NaT    NaT   None     0.0   0.236111     2155.0   
         1971-03-31    NaT    NaT   None     0.0  -0.123596     2062.0   
...                    ...    ...    ...     ...        ...        ...   
N5946510 2011-08-31    NaT    NaT   None     0.0  -0.089113  2419635.0   
         2011-09-30    NaT    NaT   None     0.0  -0.181205  1672860.0   
         2011-10-31    NaT    NaT   None     0.0   0.151854  1538552.0   
         2011-11-30    NaT    NaT   None     0.0  -0.002044  1220142.0   
         2011-12-30    NaT    NaT   None     0.0   0.098822  1185432.0   

                        PRC  SPREAD    SHROUT         MCAP  ... lag_7 lag_10  \
CUSIP    date                                               ...                
00003210 1970-11-30   9.000     NaN    2655.0    23895.000  ...   NaN    NaN   
         1970-12-31  10.000     NaN    2655.0    26550.000  ...   NaN    NaN   
         1971-01-29   9.000     NaN    2655.0    23895.000  ...   NaN    NaN   
         1971-02-26  11.125     NaN    2655.0    29536.875  ...   NaN    NaN   
         1971-03-31   9.750     NaN    2655.0    25886.250  ...   NaN    NaN   
...                     ...     ...       ...          ...  ...   ...    ...   
N5946510 2011-08-31  20.750     NaN  426263.0  8844957.250  ...   NaN    NaN   
         2011-09-30  16.990     NaN  426485.0  7245980.150  ...   NaN    NaN   
         2011-10-31  19.570     NaN  426512.0  8346839.840  ...   NaN    NaN   
         2011-11-30  19.530     NaN  426512.0  8329779.360  ...   NaN    NaN   
         2011-12-30  21.460     NaN  426678.0  9156509.880  ...   NaN    NaN   

                    div_yr div_semi_annual div_quarterly div_unknown  \
CUSIP    date                                                          
00003210 1970-11-30    NaN             NaN           NaN         NaN   
         1970-12-31    NaN             NaN           NaN         NaN   
         1971-01-29    NaN             NaN           NaN         NaN   
         1971-02-26    NaN             NaN           NaN         NaN   
         1971-03-31    NaN             NaN           NaN         NaN   
...                    ...             ...           ...         ...   
N5946510 2011-08-31    NaN             NaN           NaN         NaN   
         2011-09-30    NaN             NaN           NaN         NaN   
         2011-10-31    NaN             NaN           NaN         NaN   
         2011-11-30    NaN             NaN           NaN         NaN   
         2011-12-30    NaN             NaN           NaN         NaN   

                    div_monthly lag_quarterly lag_semi_annual lag_div_yr  
CUSIP    date                                                             
00003210 1970-11-30         NaN           NaN             NaN        NaN  
         1970-12-31         NaN           NaN             NaN        NaN  
         1971-01-29         NaN           NaN             NaN        NaN  
         1971-02-26         NaN           NaN             NaN        NaN  
         1971-03-31         NaN           NaN             NaN        NaN  
...                         ...           ...             ...        ...  
N5946510 2011-08-31         NaN           NaN             NaN        NaN  
         2011-09-30         NaN           NaN             NaN        NaN  
         2011-10-31         NaN           NaN             NaN        NaN  
         2011-11-30         NaN           NaN             NaN        NaN  
         2011-12-30         NaN           NaN             NaN        NaN  

[2097733 rows x 28 columns]

In [54]:
# Strategies 1: Long predicted dividend and short all other companies
crsp_temp['long_stocks'] = (crsp_temp['lag_3'] ==True or crsp_temp['lag_6'] ==True or crsp_temp['lag_9'] ==True or crsp_temp['lag_12'] ==True or crsp_temp['lag_quarterly'] ==True or crsp_temp['lag_semi_annual'] ==True or crsp_temp['lag_div_yr'] ==True  )


crsp_temp['short_stocks'] = (crsp_temp['lag_3'] ==False & crsp_temp['lag_6'] ==False & crsp_temp['lag_9'] ==False & crsp_temp['lag_12'] ==False & crsp_temp['lag_quarterly'] ==False & crsp_temp['lag_semi_annual'] ==False & crsp_temp['lag_div_yr'] ==False  )

crsp_temp

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().